In [51]:
import pandas as pd
import ast
from collections import Counter, defaultdict

import spacy

from tqdm.auto import tqdm

In [52]:
nlp = spacy.load("en_core_web_sm")

In [53]:
df = pd.read_csv("per_prompt_word_attributions_mistral_7b_base.csv")

df.head()

,PROMPT ID,PROMPT,WORDS
0,0,Howard’s confession was captured on camera thi...,"[('howard’s', 0.07027333229780197), ('confessi..."
1,1,"I am referring to the blunt, utterly uncensore...","[('to', 0.04985685274004936), ('blunt', 0.0373..."
2,2,"At that very moment, his mother walked into th...","[('that', 0.0876477062702179), ('moment', 0.08..."
3,3,"Albuquerque, NM – While protesting his innocen...","[('albuquerque', 0.11564238928258419), ('innoc..."
4,4,"The writer concedes, “Yes, I know that Trump i...","[('concedes', 0.0773833878338337), ('yes', 0.0..."


In [65]:
def parse_words(word_str):
    if pd.isna(word_str):
        return None

    cleaned = (
        word_str
        .replace("inf", "1e6") 
        .replace("nan", "0.0")
    )

    try:
        return ast.literal_eval(cleaned)
    except Exception:
        print("Failed to parse:", word_str)
        return None


df["word_attributions"] = df["WORDS"].apply(parse_words)
df.head()

,PROMPT ID,PROMPT,WORDS,word_attributions,highlighted_words
0,0,Howard’s confession was captured on camera thi...,"[('howard’s', 0.07027333229780197), ('confessi...","[(howard’s, 0.07027333229780197), (confession,...","[howard’s, confession, the, weekend, on]"
1,1,"I am referring to the blunt, utterly uncensore...","[('to', 0.04985685274004936), ('blunt', 0.0373...","[(to, 0.04985685274004936), (blunt, 0.03732691...","[to, blunt, i, uncensored, am]"
2,2,"At that very moment, his mother walked into th...","[('that', 0.0876477062702179), ('moment', 0.08...","[(that, 0.0876477062702179), (moment, 0.080832...","[that, moment, and, at, the]"
3,3,"Albuquerque, NM – While protesting his innocen...","[('albuquerque', 0.11564238928258419), ('innoc...","[(albuquerque, 0.11564238928258419), (innocenc...","[albuquerque, innocence, nm, protesting, ufc]"
4,4,"The writer concedes, “Yes, I know that Trump i...","[('concedes', 0.0773833878338337), ('yes', 0.0...","[(concedes, 0.0773833878338337), (yes, 0.07093...","[concedes, yes, trump, that, the]"


In [55]:
TOP_K = 5

def top_k_words(word_attr, k=TOP_K):
    return [w for w, score in word_attr[:k]]

df["highlighted_words"] = df["word_attributions"].apply(top_k_words)
df.head()

,PROMPT ID,PROMPT,WORDS,word_attributions,highlighted_words
0,0,Howard’s confession was captured on camera thi...,"[('howard’s', 0.07027333229780197), ('confessi...","[(howard’s, 0.07027333229780197), (confession,...","[howard’s, confession, the, weekend, on]"
1,1,"I am referring to the blunt, utterly uncensore...","[('to', 0.04985685274004936), ('blunt', 0.0373...","[(to, 0.04985685274004936), (blunt, 0.03732691...","[to, blunt, i, uncensored, am]"
2,2,"At that very moment, his mother walked into th...","[('that', 0.0876477062702179), ('moment', 0.08...","[(that, 0.0876477062702179), (moment, 0.080832...","[that, moment, and, at, the]"
3,3,"Albuquerque, NM – While protesting his innocen...","[('albuquerque', 0.11564238928258419), ('innoc...","[(albuquerque, 0.11564238928258419), (innocenc...","[albuquerque, innocence, nm, protesting, ufc]"
4,4,"The writer concedes, “Yes, I know that Trump i...","[('concedes', 0.0773833878338337), ('yes', 0.0...","[(concedes, 0.0773833878338337), (yes, 0.07093...","[concedes, yes, trump, that, the]"


In [56]:
records = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    prompt = row["PROMPT"]
    highlighted = set(row["highlighted_words"])

    doc = nlp(prompt)

    for token in doc:
        if token.text.lower() in highlighted:
            records.append({
                "prompt_id": row["PROMPT ID"],
                "word": token.text.lower(),
                "lemma": token.lemma_,
                "pos": token.pos_,
                "dep": token.dep_,
                "head": token.head.text.lower()
            })


  0%|          | 0/216 [00:00<?, ?it/s]

In [57]:
syntax_df = pd.DataFrame(records)

syntax_df.head()


,prompt_id,word,lemma,pos,dep,head
0,0,confession,confession,NOUN,nsubjpass,captured
1,0,on,on,ADP,prep,captured
2,0,weekend,weekend,NOUN,npadvmod,captured
3,0,the,the,DET,det,student
4,1,i,I,PRON,nsubj,referring


In [58]:
pos_counts = syntax_df["pos"].value_counts(normalize=True)
pos_counts


pos
NOUN     0.167946
PRON     0.159309
VERB     0.127639
ADP      0.099808
DET      0.091171
PROPN    0.066219
ADJ      0.064299
SCONJ    0.045106
PART     0.040307
ADV      0.039347
CCONJ    0.037428
AUX      0.029750
INTJ     0.017274
NUM      0.010557
X        0.003839
Name: proportion, dtype: float64

In [59]:
dep_counts = syntax_df["dep"].value_counts(normalize=True)
dep_counts

dep
nsubj        0.145873
prep         0.095010
det          0.092131
ROOT         0.082534
dobj         0.067179
pobj         0.062380
amod         0.052783
advmod       0.048944
aux          0.046065
compound     0.038388
cc           0.035509
mark         0.028791
ccomp        0.022073
attr         0.019194
conj         0.016315
xcomp        0.016315
poss         0.015355
appos        0.015355
advcl        0.015355
intj         0.011516
punct        0.007678
nsubjpass    0.007678
npadvmod     0.007678
acomp        0.005758
relcl        0.005758
prt          0.004798
nummod       0.004798
dep          0.003839
oprd         0.003839
acl          0.002879
pcomp        0.001919
dative       0.001919
parataxis    0.001919
preconj      0.001919
auxpass      0.001919
meta         0.001919
csubj        0.001919
neg          0.001919
predet       0.000960
nmod         0.000960
quantmod     0.000960
Name: proportion, dtype: float64

In [73]:
spacy.explain('pobj')

'object of preposition'

In [62]:
syntax_df[syntax_df["pos"] == "VERB"]["dep"].value_counts(normalize=True)

dep
ROOT         0.443609
ccomp        0.127820
xcomp        0.105263
advcl        0.097744
conj         0.052632
relcl        0.037594
compound     0.030075
acl          0.022556
amod         0.015038
parataxis    0.015038
prep         0.007519
csubj        0.007519
intj         0.007519
pobj         0.007519
appos        0.007519
acomp        0.007519
oprd         0.007519
Name: proportion, dtype: float64

In [63]:
syntax_df[syntax_df["pos"] == "NOUN"]["dep"].value_counts(normalize=True)

dep
dobj         0.228571
nsubj        0.194286
pobj         0.182857
compound     0.097143
attr         0.085714
ROOT         0.045714
appos        0.045714
npadvmod     0.028571
conj         0.028571
nsubjpass    0.017143
dep          0.011429
oprd         0.011429
advmod       0.005714
dative       0.005714
advcl        0.005714
poss         0.005714
Name: proportion, dtype: float64

In [64]:
syntax_df[syntax_df["pos"] == "PRON"]["dep"].value_counts(normalize=True)

dep
nsubj        0.602410
dobj         0.114458
pobj         0.114458
poss         0.066265
attr         0.024096
punct        0.018072
nsubjpass    0.012048
appos        0.012048
ROOT         0.012048
det          0.006024
conj         0.006024
nmod         0.006024
advcl        0.006024
Name: proportion, dtype: float64